# Modular EMA/SMA Strategy Demo

This notebook shows how to configure and exercise the trading components
from this repository without connecting to Alpaca. It relies on the
in-memory broker and synthetic data generators from `example_simulation.py`
so you can tweak parameters such as symbols, indicator periods, and cash
allocation to observe the resulting orders.

## 1. Import the modular building blocks

The configuration dataclasses describe the strategy inputs, while the
indicator, risk, and strategy classes orchestrate the trading logic. The
`FakeBroker` and helpers imported from `example_simulation` stand in for the
live Alpaca services during this demo.

In [ ]:
from datetime import datetime
import asyncio

from config import StrategyConfig, IndicatorConfig, AllocationConfig, RiskConfig
from indicators import IndicatorSet
from risk import RiskManager
from strategy import EmaSmaStrategy
from example_simulation import FakeBroker, generate_test_stream


## 2. Configure symbols and indicator parameters

You can swap tickers, tune the EMA/SMA lookbacks, and change allocation
rules here. These settings mirror the suggestions for turning the original
notebook into an importable module.

In [ ]:
strategy_config = StrategyConfig(
    bullish_symbol="AAPL",
    bearish_symbol="SQQQ",
    indicators=IndicatorConfig(ema_period=8, sma_period=21, atr_period=14, seed_bars=5),
    allocation=AllocationConfig(cash_fraction=0.4, order_time_in_force="day"),
    risk=RiskConfig(close_all_at=None),
)
strategy_config.stream.symbols = [strategy_config.bullish_symbol]
strategy_config

## 3. Run the offline bar stream through the strategy

The helper below seeds the indicators with the requested lookback, then
feeds the remaining synthetic bars into the EMA/SMA crossover strategy.
The fake broker records resulting orders so you can inspect them.

In [ ]:
async def run_offline_demo(config: StrategyConfig):
    broker = FakeBroker(config, buying_power=5_000)
    indicators = IndicatorSet.from_config(
        ema_period=config.indicators.ema_period,
        sma_period=config.indicators.sma_period,
        atr_period=config.indicators.atr_period,
    )
    risk_manager = RiskManager(broker, config.risk)
    strategy = EmaSmaStrategy(broker, indicators, risk_manager)

    bars = generate_test_stream(datetime(2023, 1, 3, 9, 30))
    seed = bars[: config.indicators.seed_bars]
    live = bars[config.indicators.seed_bars :]

    indicators.seed_from_bars(seed)

    for bar in live:
        await strategy.on_bar(bar)

    return broker

demo_broker = asyncio.run(run_offline_demo(strategy_config))
demo_broker.orders


## 4. Inspect the generated orders and final positions

The fake broker keeps an in-memory ledger so you can explore the strategy's
decisions after the synthetic stream completes.

In [ ]:
for order in demo_broker.orders:
    print(f"{order.side:>4} {order.qty:>4} {order.symbol} (TIF={order.tif})")

print("\nFinal simulated positions:")
for symbol, qty in demo_broker.positions.items():
    print(f"  {symbol}: {qty} shares")
